This script reads a plan(with multiple line details for multiple days) and a schedule from an excel file and adds/ overwrite a target_kpi file considering the hourly kpi distribution semantically.

In [ ]:
!pip install pandas openpyxl

In [ ]:
# all imports
import pandas as pd
import numpy as np
import random
from random import randint
from google.colab import drive

In [ ]:
# Read plan and schedule data from drive
drive.mount('/content/drive')
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]

NameError: name 'drive' is not defined

In [ ]:
plan.head()

,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,1100,660,23,5,0,0,28,8.28,9108
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1000,660,23,5,0,0,28,9.45,9450
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,800,660,20,5,0,0,25,8.28,9108
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,1000,660,23,5,0,0,28,10.11,10110
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,900,660,23,5,0,0,28,8.21,7389


In [ ]:
schedule

,hour,start_time,end_time
0,1,07:30:00,08:30:00
1,2,08:30:00,09:30:00
2,3,09:30:00,10:30:00
3,4,10:30:00,11:30:00
4,5,11:30:00,13:00:00
5,6,13:00:00,14:00:00
6,7,14:00:00,15:00:00
7,8,15:00:00,16:00:00
8,9,16:00:00,17:00:00
9,10,17:00:00,18:00:00


In [ ]:
# Generate the skeleton of the target table
repeated_plan_date =  plan['date'].repeat(len(schedule)).reset_index(drop=True)
repeated_module_id = plan['module_id'].repeat(len(schedule)).reset_index(drop=True)
hours = list(range(len(schedule)+1))
hours.remove(0)
repeated_hour = hours*len(plan)
targets = pd.DataFrame({'Date': repeated_plan_date, 'Line' : repeated_module_id,
                        'Hour': repeated_hour})




In [ ]:
targets

,Date,Line,Hour
0,2024-07-26,F1M7,1
1,2024-07-26,F1M7,2
2,2024-07-26,F1M7,3
3,2024-07-26,F1M7,4
4,2024-07-26,F1M7,5
...,...,...,...
149,2024-08-01,F1M8,7
150,2024-08-01,F1M8,8
151,2024-08-01,F1M8,9
152,2024-08-01,F1M8,10


In [ ]:
random.seed(42)

In [ ]:

def distribute_total(total_qty, num_hours):
  """
    Distribute a given number of total qty between a specified number
    of hours.

    :param total_qty: The total qty to be distributed.
    :param num_hours: The number of hours.
    :return: A list with the distributed values.
  """
  # Initial distribution: assign each event a base value

  base_qty = total_qty // num_hours
  remaining = total_qty % num_hours

  distribution = [base_qty] * num_hours

  # Randomly distribute the remaining value
  for _ in range(remaining):
      distribution[np.random.randint(0, num_hours)] += 1

  return distribution

In [ ]:
def generate_hourly_counts(targets, plan, line_quality_regulator):
    """
    Generate counts for each hour of each day for each line group of a DataFrame
    to match the target counts by distributing the target sum among all rows in
    the hour + date + line group.

    :param target: DataFrame containing targets.
    :param plan: DataFrame with planned values per day.
    :param line_quality_regulator: Variable determining what is the number of
    maximum pcs a line fails to produce from a given daily target ,
    given as a dictionary.
    :return: DataFrame with generated counts.
    """
    # Merge the dataframes on the 'hour, date, line' columns
    merged_df = targets.merge(plan[['date', 'module_id','qty', 'mins', 'operator', 'helper',
       'ironman', 'qc', 'manpower', 'smv', 'produced_mins']],
                              left_on=['Date', 'Line'],
                              right_on= ['date','module_id'], how='left')
    merged_df.drop(columns=['date', 'module_id'], inplace=True)


    # Create a new column for generated counts
    #merged_df['generated_qty'] = 0


    # Iterate over each date, hour group
    for (date, line), group in merged_df.groupby(['Date', 'Line']):
        # Get the target sum for this date + line
        target_sum = group['qty'].iloc[0]

        # Generate the new counts by distributing the target sum
        generated_counts = distribute_total(target_sum, len(group))

        # Adjust generated counts
        adjusted_counts = [count - randint(0, line_quality_regulator[line])
        for count in generated_counts]

        merged_df.loc[group.index, 'FTT'] = adjusted_counts

    # # # Drop the 'target_count' column as it's no longer needed
    # # result_df = merged_df.drop(columns=['target_count'])

    return merged_df

In [ ]:
# Generate FTTs close(but lesser in sum) to the daily target qty
line_quality_regulator = {'F1M7': 4, 'F1M8': 10}
merged = generate_hourly_counts(targets, plan,line_quality_regulator)
merged['FTT'] = merged['FTT'].astype(int)
merged

,Date,Line,Hour,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins,FTT
0,2024-07-26,F1M7,1,1100,660,23,5,0,0,28,8.28,9108,100
1,2024-07-26,F1M7,2,1100,660,23,5,0,0,28,8.28,9108,100
2,2024-07-26,F1M7,3,1100,660,23,5,0,0,28,8.28,9108,98
3,2024-07-26,F1M7,4,1100,660,23,5,0,0,28,8.28,9108,99
4,2024-07-26,F1M7,5,1100,660,23,5,0,0,28,8.28,9108,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,800,660,14,6,0,0,20,8.28,6624,66
150,2024-08-01,F1M8,8,800,660,14,6,0,0,20,8.28,6624,71
151,2024-08-01,F1M8,9,800,660,14,6,0,0,20,8.28,6624,62
152,2024-08-01,F1M8,10,800,660,14,6,0,0,20,8.28,6624,69


In [ ]:
#Convert start_time and end_time to datetime
schedule['start_time'] = pd.to_datetime(schedule['start_time'], format='%H:%M:%S')
schedule['end_time'] = pd.to_datetime(schedule['end_time'], format='%H:%M:%S')

# Calculate the duration between start_time and end_time in hours
schedule['clock_hours'] = (schedule['end_time'] - schedule['start_time']).dt.total_seconds() / 3600

# Sum the clock_hours to get total hours in clock time
total_clock_hours = schedule['clock_hours'].sum()
total_clock_hours

11.5

In [ ]:
# Transforms for the target dataframe
merged.rename(columns={'smv': 'SMV'}, inplace=True)
merged['TPPM'] = round(merged['qty'] / ((total_clock_hours)*60),2)
merged


,Date,Line,Hour,qty,mins,operator,helper,ironman,qc,manpower,SMV,produced_mins,FTT,TPPM
0,2024-07-26,F1M7,1,1100,660,23,5,0,0,28,8.28,9108,100,1.59
1,2024-07-26,F1M7,2,1100,660,23,5,0,0,28,8.28,9108,100,1.59
2,2024-07-26,F1M7,3,1100,660,23,5,0,0,28,8.28,9108,98,1.59
3,2024-07-26,F1M7,4,1100,660,23,5,0,0,28,8.28,9108,99,1.59
4,2024-07-26,F1M7,5,1100,660,23,5,0,0,28,8.28,9108,99,1.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,800,660,14,6,0,0,20,8.28,6624,66,1.16
150,2024-08-01,F1M8,8,800,660,14,6,0,0,20,8.28,6624,71,1.16
151,2024-08-01,F1M8,9,800,660,14,6,0,0,20,8.28,6624,62,1.16
152,2024-08-01,F1M8,10,800,660,14,6,0,0,20,8.28,6624,69,1.16


In [ ]:
def generate_defective_kpis(merged, defective_quality_regulator, defect_regulator,
                            rectifying_regulator, line_wastage_regulator):
  """
  Generate defective KPIs for a given DataFrame statistically
  based on the provided parameters.

  :param merged: skeleton df for targets
  :param defective_quality_regulator: Maximum number of defective possible
  for the line of concern
  :param defect_regulator: Maximum difference between defectives and defects
  possible for the line of concern
  :param rectifying_regulator: Maximum pcs that would not be possible to be
  rectified froma given defective count for the line of concern
  param line_wastage_regulator: Maximum pcs that are not considred defective
  that will be rejected starightaway from the line of concern
  :return: None

  """
  merged.loc[:, 'Defective'] = np.random.randint(0,defective_quality_regulator,
                                                 len(merged))
  merged.loc[merged['Defective'] == 0, 'Defect'] = 0
  merged.loc[merged['Defective'] == 0, 'Rectified'] = 0

  merged.loc[merged['Defective'] > 0, 'Defect'] = merged.loc[merged['Defective'] > 0,
                                                             'Defective'] + np.random.randint(0,defect_regulator,
                                                                                              len(merged[merged['Defective'] > 0]))
  merged.loc[merged['Defective'] > 0, 'Rectified'] = merged.loc[merged['Defective'] > 0,
                                                                'Defective'] - np.random.randint(0,rectifying_regulator, len(merged[merged['Defective'] > 0]))
  merged.loc[merged['Rectified'] < 0, 'Rectified'] = 0

  merged.loc[:, 'Rejet'] = merged['Defective'] - merged['Rectified'] + np.random.randint(0,line_wastage_regulator, len(merged))
  return merged

In [ ]:
defective_quality_regulator = {'F1M7': 10, 'F1M8': 10}
defect_regulator = {'F1M7': 4, 'F1M8': 5}
rectifying_regulator = {'F1M7': 4, 'F1M8': 3}
line_wastage_regulator = {'F1M7': 3, 'F1M8': 2}
modules = merged['Line'].unique()
new_merged = pd.DataFrame()
for module in modules:
  modulewise_df = merged[merged['Line']== module]
  result = generate_defective_kpis(modulewise_df, defective_quality_regulator[module],
                          defect_regulator[module],
                          rectifying_regulator[module],
                          line_wastage_regulator[module]
                          )
  new_merged = pd.concat([new_merged, result])
  new_merged.reset_index(drop=True, inplace=True)
new_merged


<ipython-input-138-e9a3f05cdb56>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.loc[:, 'Defective'] = np.random.randint(0,defective_quality_regulator,
<ipython-input-138-e9a3f05cdb56>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.loc[merged['Defective'] == 0, 'Defect'] = 0
<ipython-input-138-e9a3f05cdb56>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

,Date,Line,Hour,qty,mins,operator,helper,ironman,qc,manpower,SMV,produced_mins,FTT,TPPM,Defective,Defect,Rectified,Rejet
0,2024-07-26,F1M7,1,1100,660,23,5,0,0,28,8.28,9108,100,1.59,9,9.0,9.0,1.0
1,2024-07-26,F1M7,2,1100,660,23,5,0,0,28,8.28,9108,100,1.59,8,11.0,6.0,4.0
2,2024-07-26,F1M7,3,1100,660,23,5,0,0,28,8.28,9108,98,1.59,2,5.0,1.0,1.0
3,2024-07-26,F1M7,4,1100,660,23,5,0,0,28,8.28,9108,99,1.59,0,0.0,0.0,2.0
4,2024-07-26,F1M7,5,1100,660,23,5,0,0,28,8.28,9108,99,1.59,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,800,660,14,6,0,0,20,8.28,6624,66,1.16,1,4.0,0.0,2.0
150,2024-08-01,F1M8,8,800,660,14,6,0,0,20,8.28,6624,71,1.16,7,8.0,5.0,3.0
151,2024-08-01,F1M8,9,800,660,14,6,0,0,20,8.28,6624,62,1.16,2,2.0,2.0,0.0
152,2024-08-01,F1M8,10,800,660,14,6,0,0,20,8.28,6624,69,1.16,8,11.0,8.0,0.0


In [ ]:
new_merged['Defect'] = new_merged['Defect'].astype(int)
new_merged['Rectified'] = new_merged['Rectified'].astype(int)
new_merged['Reject'] = new_merged['Rejet'].astype(int)

In [ ]:
def generate_manpower_kpis(merged, manpower_regulator):
  """
  Generate manpower KPIs for a given DataFrame statistically
  based on the provided parameters.

  :param merged: skeleton df for targets
  :param manpower_regulator: Maximum number of operator/helper variation possible
  :return: None

  """
  merged['Operator'] = (merged['operator'] - np.random.choice([0,manpower_regulator],
                                          len(merged), p=[0.95, 0.05])).astype(int)
  merged['Helper'] = (merged['helper'] - np.random.choice([0,manpower_regulator],
                                          len(merged), p=[0.95, 0.05])).astype(int)
  merged['Man power'] = merged['Operator'] + merged['Helper']
  merged.drop(columns=['operator', 'helper', 'ironman', 'qc','manpower', 'qty', 'mins','produced_mins'], inplace=True)
  return merged

In [ ]:
new_merged = generate_manpower_kpis(new_merged, manpower_regulator= 1)

In [ ]:
new_merged = new_merged [['Date', 'Line', 'Hour', 'FTT', 'SMV', 'TPPM' , 'Defective' ,
                 'Defect' , 'Rectified' ,'Reject' , 'Operator' ,'Helper' ,
                 'Man power']]

new_merged['Date'] = new_merged['Date'].dt.strftime('%Y-%m-%d')




<ipython-input-144-014620b1d8f1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_merged['Date'] = new_merged['Date'].dt.strftime('%Y-%m-%d')


In [ ]:
new_merged

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Man power
0,2024-07-26,F1M7,1,100,8.28,1.59,9,9,9,1,23,5,28
1,2024-07-26,F1M7,2,100,8.28,1.59,8,11,6,4,23,5,28
2,2024-07-26,F1M7,3,98,8.28,1.59,2,5,1,1,23,5,28
3,2024-07-26,F1M7,4,99,8.28,1.59,0,0,0,2,23,4,27
4,2024-07-26,F1M7,5,99,8.28,1.59,0,0,0,0,23,5,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,66,8.28,1.16,1,4,0,2,14,6,20
150,2024-08-01,F1M8,8,71,8.28,1.16,7,8,5,3,14,6,20
151,2024-08-01,F1M8,9,62,8.28,1.16,2,2,2,0,14,6,20
152,2024-08-01,F1M8,10,69,8.28,1.16,8,11,8,0,14,6,20


In [ ]:
new_merged.to_excel('synthetic_targets.xlsx', index=False)


df_from_new_excel = pd.read_excel('synthetic_targets.xlsx')

# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    df_from_new_excel.to_excel(writer, sheet_name='target_kpis', index=False)